### Librairies

In [17]:
import os
import pandas as pd
import requests

### Authentification pour récupérer le token

In [21]:
headers_auth = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data_auth = {
    "grant_type": "client_credentials",
    "client_id": os.getenv("SPOTIFY_CLIENT_ID"),
    "client_secret": os.getenv("SPOTIFY_CLIENT_SECRET")
}

url_auth = "https://accounts.spotify.com/api/token"

try:
    response_auth = requests.post(url_auth, headers=headers_auth, data=data_auth)
    response_auth.raise_for_status()  # lève une exception si statut != 200

    # Extraction du token
    access_token = response_auth.json().get("access_token")

except requests.exceptions.RequestException as error:
    print("Exception during authentication:", error)
    access_token = None

In [22]:
headers_auth = {"Authorization": f"Bearer {access_token}"}

artist_id = "7jy3rLJdDQY21OgRLCZ9sD"  # Foo Fighters

### Appel API Artist / Albums / Tracks

In [26]:
if access_token:

    # ARTIST
    print("Get artist informations ... ")

    url_artist = f"https://api.spotify.com/v1/artists/{artist_id}"

    try:
        response_artist = requests.get(url_artist, headers=headers_auth)
        response_artist.raise_for_status()

        artist_json = response_artist.json()

        artist_data = {
            "id": artist_json["id"],
            "name": artist_json["name"],
            "url": artist_json["external_urls"]["spotify"],
            "image_url": artist_json["images"][0]["url"],
            "popularity": artist_json["popularity"],
            "genres": artist_json["genres"],  # laisse en liste
            "total_followers": artist_json["followers"]["total"],
            "uri": artist_json["uri"]
        }

        # Création du DataFrame
        df_artist = pd.DataFrame([artist_data])

        # Sauvegarde CSV
        df_artist.to_csv("../datas/artist.csv", index=False)

        print("... Artist OK")


        # ALBUMS
        print("Get albums informations ... ")

        params_albums = {
            "include_groups": "album",
            "limit": 50,
            "offset": 0
        }

        url_albums = f"https://api.spotify.com/v1/artists/{artist_id}/albums"

        try:
            response_albums = requests.get(url_albums, headers=headers_auth, params=params_albums)
            response_albums.raise_for_status()

            albums_json = response_albums.json()

            if albums_json['next'] is None:
                print("Total number of albums :", albums_json['total'])
                
                # On initialise une liste pour stocker les lignes du DataFrame
                albums_data = []
                tracks_data = []

                for album in albums_json["items"]:
                    data = {
                        "artist_id": artist_id,
                        "id": album["id"],
                        "name": album["name"],
                        "release_date": album["release_date"],
                        "release_date_precision": album["release_date_precision"],
                        "total_tracks": album["total_tracks"],
                        "url": album["external_urls"]["spotify"],
                        "image_url": album["images"][0]["url"],
                        "available_markets": album["available_markets"],
                        "uri": album["uri"]
                    }
                    albums_data.append(data)

                    # TRACKS
                    print("Get tracks informations ... ")

                    params_tracks = {
                        "limit": 50,
                        "offset": 0
                    }

                    url_tracks = f"https://api.spotify.com/v1/albums/{album["id"]}/tracks"
                    
                    try:
                        response_tracks = requests.get(url_tracks, headers=headers_auth, params=params_tracks)
                        response_tracks.raise_for_status()

                        tracks_json = response_tracks.json()

                        if tracks_json['next'] is None:
                            print("Album :", album["name"], "Total number of tracks :", tracks_json['total'])
                            
                            for track in tracks_json["items"]:
                                
                                #url_track_audio_features = f"https://api.spotify.com/v1/audio-features/{track["id"]}"

                                #try:
                                #    response_track_audio_features = requests.get(url_track_audio_features, headers=headers_auth)
                                #    response_track_audio_features.raise_for_status()

                                #    track_audio_features = response_track_audio_features.json()

                                    data = {
                                        "artist_id": artist_id,
                                        "album_id": album["id"],
                                        "album_name": album["name"],
                                        "id": track["id"],
                                        "name": track["name"],
                                        "disc_number": track["disc_number"],
                                        "track_number": track["track_number"],
                                        "duration_ms": track["duration_ms"],
                                        "explicit": track["explicit"],
                                        "url": track["external_urls"]["spotify"],
                                        "available_markets": track["available_markets"],
                                        "uri": track["uri"],
                                #        "acousticness": track_audio_features["acousticness"],
                                #        "danceability": track_audio_features["danceability"],
                                #        "energy": track_audio_features["energy"],
                                #        "instrumentalness": track_audio_features["instrumentalness"],
                                #        "key": track_audio_features["key"],
                                #        "liveness": track_audio_features["liveness"],
                                #        "loudness": track_audio_features["loudness"],
                                #        "mode": track_audio_features["mode"],
                                #        "speechiness": track_audio_features["speechiness"],
                                #        "tempo": track_audio_features["tempo"],
                                #        "time_signature": track_audio_features["time_signature"],
                                #        "valence": track_audio_features["valence"]
                                    }
                                    tracks_data.append(data)

                                #except requests.exceptions.RequestException as error:
                                #    print("Exception during tracks audio features data retrieval:", error)

                            # Création du DataFrame Tracks
                            df_tracks = pd.DataFrame(tracks_data)

                            # Sauvegarde CSV Tracks
                            df_tracks.to_csv("../datas/tracks.csv", index=False)

                            print("... OK")

                    except requests.exceptions.RequestException as error:
                        print("Exception during tracks data retrieval:", error)

                # Création du DataFrame Albums
                df_albums = pd.DataFrame(albums_data)

                # Sauvegarde CSV Albums
                df_albums.to_csv("../datas/albums.csv", index=False)

            print("... Albums OK")

        except requests.exceptions.RequestException as error:
            print("Exception during albums data retrieval:", error)

    except requests.exceptions.RequestException as error:
        print("Exception during artist data retrieval:", error)    

Get artist informations ... 
... Artist OK
Get albums informations ... 
Total number of albums : 14
Get tracks informations ... 
Album : But Here We Are Total number of tracks : 10
... OK
Get tracks informations ... 
Album : Dee Gees / Hail Satin - Foo Fighters / Live Total number of tracks : 10
... OK
Get tracks informations ... 
Album : Medicine At Midnight Total number of tracks : 9
... OK
Get tracks informations ... 
Album : Medicine At Midnight (Track Commentary) Total number of tracks : 9
... OK
Get tracks informations ... 
Album : Concrete and Gold Total number of tracks : 11
... OK
Get tracks informations ... 
Album : Sonic Highways Total number of tracks : 8
... OK
Get tracks informations ... 
Album : Wasting Light Total number of tracks : 11
... OK
Get tracks informations ... 
Album : Echoes, Silence, Patience & Grace Total number of tracks : 12
... OK
Get tracks informations ... 
Album : Skin And Bones (Live) Total number of tracks : 15
... OK
Get tracks informations ... 
Al